# 7. Transformer

## 강의 소개

- 이번 강의에서는 현재 NLP 연구 분야에서 가장 많이 활용되고 있는 Transformer(Self-Attention)에 대해 자세히 알아봅니다.
- Self-Attention은 RNN 기반 번역 모델의 단점을 해결하기 위해 처음 등장했습니다.
- RNN과 Attention을 함께 사용했던 기존과는 달리 Attention 연산만을 이용해 입력 문장/단어의 representation을 학습을 하며 좀 더 parallel한 연산이 가능한 동시에 학습 속도가 빠르다는 장점을 보였습니다

<br>

## Further Reading

- [Attention is all you need, NeurIPS'17](https://arxiv.org/abs/1706.03762)
- [Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
- [Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention.html)
- [Group Normalization](https://openaccess.thecvf.com/content_ECCV_2018/papers/Yuxin_Wu_Group_Normalization_ECCV_2018_paper.pdf)

<br>

## Further Question

- Attention은 이름 그대로 어떤 단어의 정보를 얼마나 가져올 지 알려주는 직관적인 방법처럼 보입니다.
- Attention을 모델의 Output을 설명하는 데에 활용할 수 있을까요?
- 참고
  - [Attention is not explanation](https://arxiv.org/pdf/1902.10186.pdf)
  - [Attention is not not explanation](https://www.aclweb.org/anthology/D19-1002.pdf)

<br>

## References

- Illustrated transformer
  - http://jalammar.github.io/illustrated-transformer/
- The Annotated Transformer
  - http://nlp.seas.harvard.edu/2018/04/03/attention.html
- CS224n –Deep Learning for Natural Language Processing
  - http://web.stanford.edu/class/cs224n/
- Convolution Sequence to Sequence
  - https://arxiv.org/abs/1705.03122
- Fully-parallel text generation for neural machine translation
  - https://blog.einstein.ai/fully-parallel-text-generation-for-neural-machine-translation/


<br>

## 7.1 Transformer: High-Level View

- Attention is all you need, NeurIPS’17
  - No more RNN or CNN modules


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15E_Jt0eftyRQ2-kTn-7-nm52zbiF2kkm' width=800/>

- 출처: http://jalammar.github.io/illustrated-transformer/

<br>

## 7.2 RNN: Long-Term Dependency

- input sequence: "I go home"
- 왼쪽 단어부터 오른쪽에 있는 단어들에 대해 정보를 축적해 나간다.
- attention을 사용했을 때 인코더 레벨에서 각각의 입력 단어에 대한 hidden state vector가 각 단어를 적절하게 인코딩한 벡터로서 attention의 재료로 사용된다.
- "home"이라는 단어를 인코딩한 hidden state vector는 "home"에 대한 정보를 주로 담고 있고 나머지 단어 "I", "go"에 대한 정보도 일부 포함하고 있다.
- 각 단어에 대한 정보가 hidden state vector에 어떻게 담겨있는 가를 아래 그림의 오른쪽 상단 화살표를 통해 알 수 있다.
- "home"에 대한 hidden state vector가 멀리 떨어진 "I"라는 단어의 정보를 적절히 포함하고 있으려면 RNN의 time step 갯수만큼 통과를 해서 전달이 된다.
- 이 경우 멀리있는 정보를 전달받기 때문에 gradient vanishing/exploding, long term dependency 와 같은 문제가 발생할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15FuOeWaxYaZFFBDl3VwA_ksMuHoHiK98' width=800/>

<br>

## 7.3 Bi-Directional RNNs

- 단방향 RNN을 사용하게 되면 왼쪽에 나타나는 단어에 대한 hidden state vector는 오른쪽에서 나타난 단어에 대한 정보를 담을 수 없다.
- 주어진 시퀀스에 대해서 forward 방향과 backward 방향으로 인코딩하는 방식도 생각해볼 수 있다.
- Forward RNN의 파라미터와는 독립된 별도의 파라미터를 갖는 Backward RNN 모듈을 구성한다.
- Backward RNN에서는 "I"라는 단어의 hidden state vector는 오른쪽에서 입력된 단어들인 "home", "go"에 대한 정보를 담을 수 있게 된다.
- ex) "go"라는 단어의 경우
  - Forward RNN에서는 "go"의 왼쪽에서 등장했던 단어들의 정보를 담고 있는 hidden state vector를 만들어낼 수 있다. (이를 2차원 벡터($h_2^f$)라고 가정)
  - Backward RNN에서는 "go"의 오른쪽에서 등장했던 단어들의 정보를 담고 있는 hidden state vector를 만들어낼 수 있다. (이를 2차원 벡터($h_2^b$)라고 가정)
- 각 단어별로 인코딩 벡터를 구할 때 forward RNN과 backward RNN 모듈을 각각 병렬적으로 만든 후 각 모듈에서 나타난 특정 time step의 hidden state vector들($h_2^f, h_2^b$)을 concat해줌으로서 hidden state vector의 2배에 해당하는 특정 단어에 대한 인코딩 벡터를 만들 수 있다.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15JKhOXsoOuQRosQMSFDuPOII17QMJo8L' width=600/>

<br>

## 7.4 Transformer: Long-Term Dependency

- Transformer에서 제안된 Attention 모듈의 구조와 작동방식에 대해 알아보자.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15LtcQp6oBhMLsQz-7oH474d2DODucqi3' width=800/>

- 출처: http://jalammar.github.io/illustrated-transformer/

<br>

### 7.4.1 Output Vectors

- "I", "go", "home" 이라는 단어 3개로 이루어진 시퀀스가 입력으로 주어졌다고 하자.
- 각 입력 단어 3개에 대한 input vector들이 attention 모듈을 통과했을 때 입력으로 주어진  시퀀스의 전체 내용을 잘 반영하고 인코딩한 출력 벡터들($h_1, h_2, h_3$)이 나오게 된다. 
  - 이런 측면에서는 RNN에서 각각의 입력 단어들에 대한 정보를 담고 있는 hidden state vector를 만들어내는 것과 동일하다고 볼 수 있다.
  - Attention 모듈에서도 입력과 출력의 세팅은 동일하게 유지된다는 것을 알 수 있다.

<br>

### 7.4.2 Self-Attention

- Attention이 적용되는 방식은 seq2seq w/ attention의 방식과 유사하다.
- "I" 라는 단어에 대한 인코딩 벡터를 만든다고 가정하자.
- Transformer의 attention 모듈에서는 "I" 라는 단어에 대한 input vector($x_1$)가 seq2seq w/ attention에서 디코더의 특정 time step의 hidden state vector와 같은 역할을 하면서 attention 모듈의 입력으로 들어간다.
- attention 모듈의 또 다른 입력으로 사용되는 인코더의 각 time step에서의 hidden state vector의 역할은 입력 시퀀스의 모든 단어에 대한 입력 벡터들($x_1, x_2, x_3$)이 그 역할을 하게 된다.
- (정리) "I" 라는 단어에 대한 인코딩 벡터를 만들기 위한 Transformer의 Attention 모듈의 입력
  - <font color='skyblue'>$x_1$: 디코더의 hidden state vector 역할</font>
  - <font color='skyblue'>$x_1, x_2, x_3$: 인코더의 각 time step의 hidden state vector 역할</font>
- $x_1$는 자기자신과 한번 내적이 수행되고, $x_2$와 $x_3$에 대해 각각 내적이 수행되므로서 주어진 3개의 벡터와의 내적에 기반한 유사도를 구하게 된다.
- 이렇게 구해진 유사도에 softmax를 취해 확률값으로 표현된 가중치를 얻을 수 있다.
- 이 가중치를 입력벡터들과 가중합하여 입력 단어 "I"에 대한 인코딩 벡터를 생성한다.
- 위와 같은 과정을 "go"와 "home"에 대해서도 수행하여 각각의 인코딩 벡터를 생성할 수 있다.
  - $h_1 = Attention(x_1, \{x_1, x_2, x_3\})$
  - $h_2 = Attention(x_2, \{x_1, x_2, x_3\})$
  - $h_3 = Attention(x_3, \{x_1, x_2, x_3\})$
- 디코더의 hidden state vector와 인코더의 hidden state vector들간의 유사도를 구할 때 동일한 세트의 벡터들에 대해서 적용이 된다는 측면에서 이를 <font color='skyblue'>self-attention 모듈</font>이라고 부른다.

<br>

- 자기자신과의 내적을 통해 유사도를 구하게 되면 다른 벡터와 내적을 했을 때보다 더 큰 유사도를 갖게 된다.
- 이렇게 되면 인코딩한 결과 벡터들도 자기자신의 정보만을 주로 포함하는 벡터가 나오게 될 것이다.
- 이러한 문제를 해결하기 위해 attention 모듈의 구조를 좀 더 유연하게 개선한다.


<br>

### 7.4.3 Query, Key, Value

- attention 모듈 내에서 주어진 입력 벡터들이 그때그때마다 서로 다른 역할을 수행한다.

<br>

#### 7.4.3.1 Query 벡터

- 특정 단어에 대한 인코딩 벡터를 얻어내기 위해 **해당 단어의 입력 벡터**는 seq2seq w/ attention에서의 디코더의 hidden state vector와 같이 주어진 입력 벡터 세트에 대해 유사도를 구하는 역할을 수행하게 된다.
- 주어진 벡터들 중에 어느 벡터를 선별적으로 가져올지에 대한 기준이 되는 벡터로서 역할을 수행하는 것을 <font color='skyblue'>Query 벡터</font>라고 한다.

<br>

#### 7.4.3.2 Key 벡터

- Query 벡터와 내적이 수행되어 유사도를 계산할 때 재료가 되는 벡터들을 <font color='skyblue'>Key 벡터</font>라고 부른다.
- 즉, 주어진 Query 벡터를 통해서 여러 개의 Key 벡터들 중에 어느 Key 벡터가 높은 유사도를 갖고 있는 지에 대한 정보를 만들게 된다.

<br>

#### 7.4.3.3 Value 벡터

- Query 벡터와 각각의 Key 벡터들간의 내적을 통해 유사도를 구한 후 softmax를 취하여 구해진 가중치들과 선형결합이 되는 대상의 벡터들 또한 입력 벡터들을 통해 만들어지게 된다.
- 가중평균을 구할 때 재료 벡터로서 사용되는 변환된 입력 벡터를 <font color='skyblue'>Value 벡터</font>라고 부른다.

<br>

#### 7.4.3.4 Linear Transformation Matrix

- 위와 같이 하나의 시퀀스를 Attention을 통해 인코딩하는 과정에서 각각의 입력 벡터들이 Query, Key, Value 의 3가지 역할을 모두 수행하게 된다.
- Transformer에서 제안된 Self-Attention 모듈은 동일한 세트의 벡터에서 출발하여 각각의 역할에 따라 벡터가 서로 다른 형태로 변환할 수 있게 해주는 linear transformation matrix를 각각 따로 갖게 된다.
  - $W^Q$: 입력 벡터를 Query 벡터로 변환해주는 matrix
  - $W^K$: 입력 벡터를 Key 벡터로 변환해주는 matrix
  - $W^V$: 입력 벡터를 Value 벡터로 변환해주는 matrix
- 주어진 matrix를 통해 같은 벡터에 대해서도 각각의 역할로 확장된 벡터를 만들 수 있게 된다.

<br>

### 7.4.4 Attention Module Process

- "I" 라는 단어를 인코딩한다고 해보자.
- "I" 라는 단어에 대한 입력 벡터 $x_1$이 $W^Q$에 의해 Query 벡터 $q_1$으로 변환된다.
- 각 단어에 대한 입력 벡터들($x_1, x_2, x_3$)은 각각 $W^K$에 의해 Key 벡터들($k_1, k_2, k_3$)로 변환되고, $W^V$에 의해 Value 벡터들($v_1, v_2, v_3$)로 변환된다.
- Key 벡터들과 Value 벡터들은 1:1로 매칭된다.
  - Query 벡터는 하나로 고정되어 있다 하더라도 Key 벡터와 Value 벡터의 갯수는 일치해야 한다.
- Query 벡터와 Key 벡터들 사이의 Attention 연산과정을 통해서 내적값이 구해진다.
- 이 내적값들을 softmax를 통과시켜서 확률값으로 바꿔준다.
  - 이 확률값을 상대적인 가중치로 볼 수 있다.
- 이렇게 구해진 가중치를 Value 벡터의 가중 평균을 구함으로서 최종적인 인코딩 벡터 $h_1$이 나오게 된다.

<br>

- 위와 같은 과정을 시퀀스 내의 모든 단어 벡터에 대해 실시하여 모든 단어에 대한 인코딩 벡터들을 만들어 낸다.
  - Key 벡터와 Value 벡터는 그대로 사용되고 Query 벡터만 해당하는 단어를 변환시켜 사용한다.

<br>

### 7.4.5 입력 벡터를 행렬로 쌓아서 연산 수행

- 4차원의 크기를 갖는 입력 벡터 2개가 있다고 하자.
  - "Thinking", "Machines"
- 이 2개의 벡터를 행단위로 결합하여 2x4 크기의 입력 행렬을 만들 수 있다.
- 여기에 크기가 4x3인 각각 Query, Key, Value 벡터로 변환하는 W 행렬과 행렬곱 연산을 수행하여 2x3 크기의 Query, Key, Value 행렬을 구할 수 있다.
  - 만들어진 행렬의 각 행은 각각의 입력 벡터가 변환된 벡터이다.

<br>

### 7.4.6 RNN의 Long-Term Dependency 문제 해결

- 각각의 인코딩된 벡터들은 시퀀스의 모든 단어에 대한 정보를 갖고 있게 된다.
- 시퀀스의 길이가 길다고 하더라도 Self-Attention 모듈을 적용해서 시퀀스 내의 각각의 단어들에 대한 인코딩 벡터를 만들게 되면 먼 거리의 있는 단어에 대한 정보도 쉽게 가져올 수 있다.
- 이러한 점이 기존의 RNN에서 갖고 있던 한계점을 근본적으로 해결한 형태의 모듈로 볼 수 있다.

<br>

## 7.5 Scaled Dot-Product Attention

- Attention 모듈의 과정을 수식적으로 살펴보자.

### 7.5.1 Inputs

- Inputs: a query $q$ and a set of key-value $(k, v)$ pairs to an output
  - Attention 모듈의 입력으로 query 벡터와 쌍으로 이루어진 key, value 벡터가 있다.
- <font color='red'>Query, key, value</font> and output is all vectors

<br>

### 7.5.2 Output

- Output is weighted sum of values
  - Output 벡터는 value 벡터에 대한 가중 평균이다.
- Weight of each value is computed by an inner product of query and corresponding key
  - 가중 평균에 사용되는 가중치는 query 벡터와 value 벡터에 해당하는 key 벡터와의 내적값을 통해서 구해진다.
- Queries and keys have same dimensionality $d_k$, and dimensionality of value is $d_v$
  - query 벡터와 key 벡터는 내적 연산이 가능해야 하기 때문에 같은 차원($d_k$)의 벡터여야 한다.
  - value 벡터의 차원($d_v$)은 query, key 벡터의 차원과 일치하지 않아도 된다.

<br>

### 7.5.3 Attention 모듈의 output: 기본 수식

\begin{equation}
A(q, K, V)=\sum_{i} \frac{\exp \left(q \cdot k_{i}\right)}{\sum_{j} \exp \left(q \cdot k_{j}\right)} v_{i}
\end{equation}

- $A()$ : Attention module
- 입력
  - $q$: query 벡터 1개
  - $K, V$: key와 value 벡터들의 pair 집합, 각각의 벡터를 행단위로 결합한 행렬로 들어온다
- $q \cdot k_{i}$
  - query 벡터와 각각의 key 벡터들 사이의 내적
- $\exp \left(q \cdot k_{i}\right)$
  - 내적 결과에 exponential 함수 적용
- $\frac{\exp \left(q \cdot k_{i}\right)}{\sum_{j} \exp \left(q \cdot k_{j}\right)}$
  - 이 값들의 상대적인 비율 계산 (softmax)
- $\frac{\exp \left(q \cdot k_{i}\right)}{\sum_{j} \exp \left(q \cdot k_{j}\right)} v_{i}$
  - 위에서 구한 유사도 값에 매칭되는 value 벡터를 곱해줌
- $\sum_{i} \frac{\exp \left(q \cdot k_{i}\right)}{\sum_{j} \exp \left(q \cdot k_{j}\right)} v_{i}$
  - 각각의 가중치가 곱해진 value 벡터들을 모두 더해 value 벡터에 대한 가중평균으로서 output 벡터가 만들어진다.

<br>

- $A(q, K, V)$의 차원의 크기는 $d_v$가 된다.

<br>

### 7.5.4 Attention 모듈의 output: 확장된 수식

- When we have multiple queries $q$, we can stack them in a matrix $Q$
- 여기서의 확장은 query 벡터가 하나의 벡터가 아니라 다수의 query 벡터를 행렬로 만들어 입력으로 받는 형태를 의미한다.

<br>

\begin{equation}
A(Q, K, V)=\operatorname{softmax}\left(Q K^{T}\right) V
\end{equation}

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15TmhHPjW-JS_Yq6ebE7X3GHdrRJ2SaGN' width=600/>

<br>

- $Q K^T$ 수행 결과 만들어지는 $|Q|\times|K|$ 크기의 행렬은 각 행에 해당하는 query 벡터들에 대한 다른 단어 벡터들과의 유사도를 갖고 있다고 볼 수 있다.
- 이렇게 구해진 $|Q|\times|K|$ 크기의 행렬에 softmax 연산을 수행하게 되면 각 행 단위로 softmax가 적용된다. (Row-wise softmax)
- softmax가 적용된 행렬과 $|V| \times d_v$ 크기의 행렬의 행렬곱 수행 결과 만들어지는 행렬의 각각의 행 벡터는 query 벡터에 대한 Attention 모듈의 Output 벡터를 의미하게 된다.

<br>

- 이렇게 행렬 연산을 통해 다수의 query 벡터에 대한 Attention 모듈의 output 벡터를 한꺼번에 계산할 수 있다.
- 여러 개의 query 벡터에 대한 attention 모듈의 계산을 행렬 연산으로 바꿔주게 되면 행렬 연산을 병렬적으로 빠르게 계산할 수 있는 GPU를 활용하여 효율적으로 수행할 수 있다.
- <font color='skyblue'>병렬적인 행렬 연산이 가능하다는 점에서 Transformer는 기존의 RNN에 비해서 상대적으로 학습이 빠른 특성을 가지게 된다.</font>

<br>

- 실제 Transformer 논문 구현 상으론 동일한 shape으로 mapping된 Q, K, V가 사용되어 각 matrix의 shape은 모두 동일하다.

<br>

### 7.5.5 Scaled Dot-Product Attention

- Example from illustrated transformer
- $Q \times K^T$를 통해 계산된 값을 $\sqrt{d_k}$로 나눠줌으로서 Scaled Dot-Product Attention이 된다.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15VTH3fknCxDItUJZS7rL759PX93dvhGQ' width=800/>

- 출처: http://jalammar.github.io/illustrated-transformer/

<br>

### 7.5.6 $Q K^T$를 $\sqrt{d_k}$로 나눠주는 이유

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15WobK7sJjv3UOmtOErpZrnlF629r5JrS' width=200/>

- query 벡터와 key 벡터들을 내적한 결과를 $\sqrt{d_k}$로 나눠주는 것이 어떤 역할을 하는 지 생각해보자.
- $q K^T$ 내적에 참여하는 query 벡터와 key 벡터의 차원 수는 그 크기를 임의로 지정할 수 있다.

<br>

- Problem
  - As $d_k$ gets large, the variance $q^T k$ increases
    - query, key 벡터의 차원의 크기가 커질수록 query 벡터와 key 벡터의 내적된 값의 분산이 증가한다.
  - Some values inside the softmax get large
  - The softmax gets very peaked
    - 분산의 큰 값들의 softmax를 적용한 확률분포는 한쪽으로 몰리게 된다.
  - Hence, its gradient gets smaller
    - 이때 gradient vanishing이 발생할 위험이 있다.
- Solution
  - Scaled by the length of query / key vectors
    - 내적된 값을 query, key 벡터의 차원의 크기로 나눠줌으로서 scaling 할 수 있다.

<br>

- <font color='skyblue'>$d_k$가 2</font>이고, query 벡터 $q$가 `[a, b]`이고, i번째 key 벡터 $k_i$가 `[x, y]`라고 하면 내적된 결과는 $ax+by$가 된다.
- 통계적으로 볼 때, query 벡터와 key 벡터의 각각의 원소의 값들을 특정한 평균과 분산을 가지는 형태의 확률변수라고 생각해보자.
- $a, b, x, y$가 각각 통계적으로 독립인 평균이 0이고 분산이 1일 때 <font color='skyblue'>내적된 결과인 $ax+by$의 평균은 0이 되고, 분산은 2가 된다.</font>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1QVcH0QOb-ake0-3FeRqu2hIHqhe3tNia' width=900 alt='상현님 제공 증명'/>

- <font color='skyblue'>$d_k$가 100</font>일 경우, 각각의 벡터의 원소가 통계적으로 독립이고 평균이 0이고 분산이 1인 경우에는 <font color='skyblue'>내적된 결과의 평균과 분산은 각각 0과 100이 된다.</font>
- $d_k$의 크기에 따라 분산의 크기가 달라지므로 내적된 값들의 분포의 차이가 발생하게 된다.
  - $d_k=2, \sigma=\sqrt{2} \approx 1.4$ -> `[1.1, -0.8, -1.7]`
  - $d_k=100, \sigma=10$ -> `[8, -11, 7]`
- 이렇게 서로 다른 분포의 값들을 softmax를 적용하게 되면 표준편차가 클수록 softmax의 확률분포가 큰 값에 몰리는 현상이 발생하게 된다.
  - 표준편차가 작은 경우에는 softmax의 확률분포가 고르게 나타나게 된다. (like uniform distribution)
- 이와 같이 $d_k$의 크기의 차이가 softmax가 적용된 확률분포의 패턴에 차이를 만들어낼 수 있게 된다.
- <font color='skyblue'>내적값의 분산을 일정하게 유지시켜 줌으로서 학습을 안정화시켜주기 위해서 내적값에 $\sqrt{d_k}$를 나눠주는 것</font>이다.
- $d_k$의 크기로 인해 output의 확률분포가 한쪽으로 몰리지 않도록 하기 위해 scaling 해주는 것으로 보면 된다.


<br>

## 7.6 Multi-Head Attention

- Multi-Head Attention: Self-Attention 모듈을 좀 더 유연하게 확장한 것

<br>

### 7.6.1 Multi-Head Attention

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15XXw25hfhjPagrkFz9dzZwnTT0cdAzz8' width=300/>

<br>

- The input word vectors are the queries, keys and values
- In other words, the word vectors themselves select each other
- Problem of single attention
  - Only one way for words to interact with one another
- Solution
  - Multi-head atten maps $Q, K, V$ into the $h$ number of lower-dimensional spaces via $W$ matrices
- Then apply attention, then concatenate outputs and pipe through linear layer

<br>

$$
\begin{gathered}
\text { MultiHead } \left.(Q, K, V)=\text { Concat(head }_{1}, \ldots, \text { head }_{h}\right) W^{O} \qquad
\text { where head }_{i}=\text { Attention }\left(Q W_{i}^{Q}, K W_{i}^{K}, V W_{i}^{V}\right)
\end{gathered}
$$

<br>

- 하나의 Self-Attention 모듈에 Q, K, V를 입력으로 주고 각 벡터들을 선형 변환($W_Q, W_K, W_V$)을 수행한 후 각각의 query 벡터에 대해서 Key 벡터와 value 벡터에 대한 attention(value 벡터의 가중합)을 통해서 인코딩된 벡터를 만들어낸다.
- 아래의 그림과 같이 Multi-Head Attention은 동일한 Q, K, V 벡터들에 대해서 동시에 병렬적으로 여러 버전의 Attention을 수행한다는 뜻이 된다.
  - <font color='skyblue'>$W_Q, W_K, W_V$가 하나의 세트만 존재하는 것이 아닌 여러 버전의 행렬 세트가 존재</font>한다.
  - i번째 Attention Block에서는 i번째의 행렬 세트를 통해 선형 변환을 한 후 Attention을 수행하여 query 벡터들에 대한 인코딩 벡터들을 얻게 된다.
- 서로 다른 버전의 Attention Block의 갯수만큼 동일한 query 벡터들에 대한 서로 다른 버전의 인코딩 벡터들이 나오고 이들을 서로 concat한 후 linea transformation을 수행함으로서 해당 query 벡터들에 대한 인코딩 벡터들을 얻게 된다.
- 여러 버전의 Attention을 수행하기 위한 선형 변환 행렬(linear transformation matrix) 세트들을 **Head**라고 부르고, 다수 버전의 Attention을 수행한다는 의미로서 **Multi-Head** Attention이라고 한다.


<br>

### 7.6.2 Multi-Head Attention의 필요성

- 동일한 시퀀스가 주어졌을 때에도 <font color='skyblue'>특정한 query 단어에 대해서 서로 다른 기준으로 여러 측면에 대해서 정보를 추출할 필요가 있다.</font>
  - ex) 다음과 같은 문장들이 있다고 하자.
    - "I went to the school."
    - "I studied hard."
    - "I came back home."
    - "I took the rest."
  - 위와 같이 여러 문장이 주어졌을 때 각 문장들에서 첫 번째로 주어진 "I"라는 query 단어에 대한 인코딩을 수행하기 위해서는 여러가지 측면에 대해서 정보를 추출할 수 있다.
    - "I"라는 주체가 수행한 행동을 중심으로 정보를 추출
    - "I"라는 주체가 존재하던 장소의 변화에 대한 정보를 추출
- 이런 방식으로 서로 다른 측면의 정보를 병렬적으로 추출하여 합치는 형태로 Attention 모듈을 구성할 수 있다.
- 각각의 Head가 위와 같은 서로 다른 정보들을 상호 보완적으로 추출하는 역할을 수행하게 된다.

<br>

### 7.6.3 Multi-Head Attention 도식화

- Example from illustrated transformer

<br>

- 두 단어("Thinking", "Machines")로 이루어진 입력 시퀀스 $X$에 대해서 두 개의 서로다른 Attention Head가 존재한다고 하자.
- 동일한 $X$ 벡터에 대해서 각각의 Head별로 존재하는 선형 변환 행렬들을 통해서 얻어지는 Q, K, V 벡터들을 가지고 각각의 Head 내에서의 인코딩된 벡터 $Z_0, Z_1$를 얻게 된다.
- $Z_0, Z_1$의 각 행에는 입력 시퀀스의 각 단어에 대응하는 인코딩 벡터를 갖고 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15__jubY1VEiI6a-g6mveu718xcEzMHGS' width=900/>

- 이렇게 구해진 $Z_0, Z_1$를 같은 입력 단어에 대한 인코딩 벡터를 기준으로 concat을 한 형태로 구성할 수 있다.
- 아래 그림과 같이 8개의 Head가 있다고 하면 각 Head별로 3차원 형태의 출력을 얻었다면 이들을 모두 다 concat하면 차원수가 24(=3x8)인, 즉 크기가 2x24 인 행렬을 얻게 된다.
  - 2: 입력 단어의 개수
  - 24: concat된 각 head의 output 벡터의 차원수
- concat을 수행한 다음 linear transformation을 수행하게 된다.
  - 이 선형 변환은 24차원의 벡터를 우리가 원하는 차원의 벡터로 변환해주기 위해 수행된다.
  - 최종적으로 얻고 싶은 벡터의 차원수가 4라고 한다면 해당 선형 변환의 가중치 행렬($W^O$)의 크기는 24x4 가 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15aKVpr6tVTZq5nrik6R2MHSzHTzq5pMv' width=900/>

<br>

### 7.6.4 Complexity

- Self-Attention 모듈의 계산량과 메모리 요구량의 측면에서 특성을 살펴보고 기존의 RNN 기반의 시퀀스 인코딩과의 비교를 해보자.

<br>

**Notation**

- $n$ is the sequence length
  - 시퀀스의 길이
- $d$ is the dimension of representation
  - 임베딩 차원
- $k$ is the kernel size of convolutions
  - convolution의 커널 크기
- $r$ is the size of the neighborhood in restricted self-attention
  - 제한된 self-attention에서의 이웃 크기

<br>

- Maximum path lengths, per-layer complexity and minimum number of sequential operations for different layer types

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15bZMX5OEZ7uyIAZrf2McCg0_F6pECdE8' width=800/>

- Complexity per Layer
  - 연산에서 가장 큰 비중을 차지하는 부분의 복잡도
- Sequential Operations
  - 병렬로 수행될 때의 복잡도
- Maximum Path Length
  - Long-term dependency와 직접적으로 관련이 되는 부분
  - 즉, 멀리 있는 단어에 대한 정보를 받아서 인코딩을 수행할 때 필요한 복잡도

<br>

#### 7.6.4.1 Self-Attention's Complexity

- <font color='skyblue'>Self-Attention</font>의 계산 복잡도(Complexity per Layer)에서 주된 부분을 차지하는 것은 $Q K^T$ 부분, 즉 query 벡터와 Key 벡터의 내적 연산이 수행되는 부분이다.
- 시퀀스의 길이가 $n$이고, query와 key의 차원이 $d$라고 한다면 $n$개의 query와 $n$개의 key에 대한 $d$개의 차원에 대한 가능한 모든 경우의 내적값을 계산해야 한다.
  - $(n \times d) \times (d \times n)$
  - $d$개의 차원에 대한 내적은 $d$만큼의 곱셈 및 덧셈을 수반한다.
- 이것을 모든 query와 key 의 조합에 대해 수행해야 하므로 $n^2 \cdot d$ 만큼의 계산이 가장 많은 부분을 차지하게 된다.
  - <font color='skyblue'>Self-Attention's Complexity per Layer = $O\left( n^2 \cdot d \right)$</font>

<br>

- 그렇지만 위와 같은 행렬 연산은 GPU의 코어 수가 충분이 많다면 시퀀스가 아무리 길거나 차원의 크기가 아무리 크더라도 모든 계산을 GPU가 가장 특화된 행렬 연산의 병렬화를 통해 코어수가 무한정 많다는 가정하에 한 번에 계산할 수 있다.
  - <font color='skyblue'>Self-Attention's Sequential Operations = $O(1)$</font>

<br>

- Self-Attention에서는 가장 멀리 있는 단어라고 하더라도 동일한 key, value 벡터로 보기 때문에 필요한 정보를 직접적으로 한번에 가져올 수 있다.
  - <font color='skyblue'>Self-Attention's Maximum Path Length = $O(1)$</font>

<br>

#### 7.6.4.2 RNN's Complexity

- Self-Attention에 비해 <font color='skyblue'>RNN</font>의 경우는 주로 필요로 하는 계산량을 살펴보자.
- 각 time step에서 계산되는 과정을 살펴볼 때 이전 time step에서 넘어오는 $h_{t-1}$ 벡터가 다음 time step의 $h_t$의 변환 과정에 참여하게 된다.
- $h_{t-1}$의 차원이 $d$라고 하면 $d \times d$ 크기의 $W_{hh}$와의 결합을 통해 $d$ 차원의 크기를 갖는 $h_t$로 변환된다.
- 이 때 $W_{hh}$ 행렬 $d$ 차원의 크기를 갖는 각각의 행과 $d$ 차원의 크기인 $h_{t-1}$의 내적이 총 $d$번 수행된다.
- 결국에는 $d^2$만큼의 계산량이 필요하게 된다.
- 이러한 계산을 매 time step마다 순차적으로 수행해줘야 하고, time step의 갯수는 시퀀스의 길이인 $n$과 동일하기 때문에 가장 핵심이 되는 계산량은 아래와 같다고 할 수 있다.
  - <font color='skyblue'>RNN's Complexity per Layer = $O\left( n \cdot d^2 \right)$</font>

<br>

- RNN에서는 매 time step마다 행렬 연산이 수행되므로 병렬화하여 처리할 수 있는 계산량은 time step의 갯수, 즉 시퀀스의 길이 $n$에 비례한다.
  - <font color='skyblue'>RNN's Sequential Operations = $O(n)$</font>

<br>

- RNN에서는 멀리 있는 단어의 정보를 가져오기 위해서는 time step의 차이 만큼 지나서 정보를 가져와야 한다.
- 가장 마지막에 있는 단어가 가장 처음에 있는 단어의 정보를 참고하기 위해 $n$번의 RNN Layer를 통과해야 한다.
  - <font color='skyblue'>RNN's Maximum Path Length = $O(n)$</font>

<br>

#### 7.6.4.3 Self-Attention과 RNN의 Complexity per Layer를 통한 메모리 사용 요구량 비교

- 위에서 계산된 것과 같은 정보들은 forward propagation 및 backpropagation 과정에서 모두 다 메모리에 저장하고 있어야 한다.
  - forward propagation에서 발생한 정보들을 모두 메모리에 저장해놓고 있는 다음 backpropagation 과정에서 이 정보들을 사용
- $d$라는 값은 RNN이나 Self-Attention Layer를 정의할 때 임의로 정할 수 있는 하이퍼파라미터이다.
- 하지만 $n$이라는 값은 임의로 고정된 값으로 사용할 수 있는 것이 아니라 주어진 입력에 따른 가변적인 값을 갖는 형태가 된다.
- 만약 긴 시퀀스를 처리하게 될 때 Self-Attention에서는 시퀀스 길이의 제곱에 비례하는 계산량과 메모리가 요구되고, RNN에서는 그에 비해 시퀀스 길이에 1차 비례하는 계산량과 메모리가 요구된다.
- 그렇기 때문에 일반적으로 Self-Attention에서는 RNN에 비해 더 많은 메모리 요구량을 필요로 하게 된다.
  - 이는 Self-Attention에서 모든 query와 모든 key 벡터들간에 내적값들을 저장하고 있어야 하기 때문이다.

<br>

#### 7.6.4.4 Self-Attention과 RNN의 병렬화가 가능한 측면에서의 비교

- Self-Attention은 시퀀스의 길이가 아무리 길더라도 GPU의 코어 수가 충분히 많다고 한다면 모든 계산을 동시에 수행할 수 있다. ($O(1)$)
- 그렇지만 RNN의 경우에는 $h_{t-1}$을 계산해야만 다음 time step의 $h_t$를 계산할 수 있기 때문에 $h_t$를 계산하기 위해서는 $h_{t-1}$이 계산될 때까지 기다릴 수밖에 없다.
- 입력은 동시에 처리할 수 있는 형태로 주어지지만 RNN 수행과정의 특성상 hidden state vector를 수행하는 과정은 병렬화가 불가능하다.
- 따라서 RNN에서는 시퀀스의 길이에 비례하는, 병렬화가 불가능한 Sequential Operations이 필요로 하게 된다.

<br>

#### 7.6.4.5 결과 종합 비교

- 이러한 사실 때문에 일반적으로 Self-Attention 기반의 Transformer 모델은 학습은 RNN보다 훨씬 더 빠르게 진행될 수 있다.
- 그렇지만 RNN에 비해 더 많은 메모리를 필요로 하게 된다.
- Maximum Path Length 측면에서도 Self-Attention이 RNN의 Long-term dependency 문제를 근본적으로 해결한 결과를 나타내게 된다.

<br>

## 7.7 Block-Based Model

- Transformer 모델에서는 Multi-Head Attention을 핵심 모듈로 하여 이와 함께 추가적인 후처리를 진행해서 아래 그림과 같은 하나의 모듈을 구성하게 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15jwDFKHQW8MSzm-Tlci5qja34-tlfesP' width=200/>

- Each block has two **sub-layers**
  - Multi-head attention
  - Two-layer feed-forward NN (with ReLU)
- Each of these two steps also has
  - Residual connection and layer normalization
  - ${LayerNorm}(x + {sublayer}(x))$

<br>

### 7.7.1 Block Components

**Multi-Head Attention Module**

- 하나의 입력 단어 벡터가 Q, K, V 벡터로 변환되어 해당 모듈의 입력으로 들어간다.
- 이러한 형태의 변환을 각 head별로 거치고 각 head의 출력으로 나오는 인코딩 벡터들을 concat하여 output layer로 존재하는 선형변환을 통해 해당 입력 단어 벡터에 대한 인코딩 벡터가 최종적으로 출력된다.

<br>

**Residual Connection (Add)**

- Residual Connection이라 불리는 입력 단어 벡터와 Multi-Head Attention 모듈의 출력 벡터 사이의 Add 연산(operation)을 수행한다.

<br>

**Layer Normalization (Norm)**

- Residual Connection이 수행된 결과 벡터에 Layer Normalization을 수행한다.

<br>

**Feed Forward Network & Add & Norm**

- 추가적으로 FFN을 통과한 후 한번 더 Residual Connection 및 Layer Normalization을 수행한다.

<br>

### 7.7.2 Residual Connection

- Residual Connection은 Computer Vision 쪽에서 깊은 layer의 Neural Network를 만들 때 Gradient Vanishing 문제를 해결하여 학습은 안정화시키면서 layer를 쌓아감에 따라 더 높은 성능을 보일 수 있도록 하는 효과적인 모델이다.

<br>

- "I study math" 라는 시퀀스에서 각각의 단어에 대한 입력 벡터가 2차원 벡터로 구성되어 있다고 하자.
  - "I": `[1, -4]`
- Multi-head Attention 모듈을 수행한 결과로 인코딩 벡터들이 출력된다.
  - "I"'s encoding: `[2, 3]`
- 입력 벡터들과 인코딩 벡터들에 대해 Add 연산을 수행하여 각 단어들에 대한 인코딩 벡터들을 만들어낸다.
  - "I"s residual connection: `[3, -1]`

<br>

- Residual Connection을 수행함으로서 Multi-Head Attention 모듈에서는 입력 벡터와 만들고자 하는 최종 인코딩 벡터와의 차이에 해당하는 값을 만들어주게 된다.
- 이를 통해 gradient vanishing 문제를 해결하고 학습도 안정화시킬 수 있게 된다.
- Residual Connection을 적용하기 위해서는 입력 벡터의 차원과 Multi-Head Attention 모듈의 출력 벡터의 차원이 반드시 일치해야 한다.
  - Multi-Head Attention의 output layer에서 head의 수가 많아짐에 따라서 각 head의 인코딩 벡터들이 concat되면서 차원이 늘어나게 되고 늘어난 차원을 원래의 입력 벡터의 차원과 일치시켜 주기 위해 선형 변환이 실시되는 것이다.

<br>

### 7.7.3 Layer Normalization

- 딥러닝에서 다양한 Normalization 기법들이 존재한다.
  - Batch Norm, Layer Norm, Instance Norm, Group Norm
- Normalization은 주어진 샘플들에 대해서 값들의 평균을 0, 분산을 1로 만들어준 후 우리가 원하는 평균과 분산을 주입할 수 있도록 하는 선형 변환으로 이루어진다.
  - 원래 값들이 갖고 있는 평균과 분산이 어떤 값이였든 간에 표준화된 평균과 분산인 0과 1로 만들어주는 과정으로 해석할 수 있다.

<br>

- Layer normalization changes input to have zero mean and unit variance, per layer and per training point (and adds two more parameters)

<br>

$$
\mu^{l}=\frac{1}{H} \sum_{i=1}^{H} a_{i}^{l}, \quad \sigma^{l}=\sqrt{\frac{1}{H} \sum_{i=1}^{H}\left(a_{i}^{l}-\mu^{l}\right)^{2}}, \quad h_{i}=f\left(\frac{g_{i}}{\sigma_{i}}\left(a_{i}-\mu_{i}\right)+b_{i}\right)
$$

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15k5kvs_iHJOErWYos6XG45YnxpEJqCw5' width=800/>

<br>

- Layer normalization consists of two steps

<br>

- step 1
  - Normalization of each word vectors to have mean of zero and variance of one.
  - 주어진 샘플들에 대한 평균과 분산을 각각 0과 1로 만들어준다.
- step 2
  - Affine transformation of each sequence vector with learnable parameters
  - 우리가 원하는 평균과 분산을 주입한다.

<br>

**step 1 details**

- 아래 그림에서 왼쪽과 같이 "thinking"과 "machines"라는 단어에 대한 인코딩 벡터가 4차원 형태로 다음과 같이 주어졌다고 하자.
  - "thinking"에 대한 인코딩 벡터: `[4, 2, 3, 5]`
  - "machines"에 대한 인코딩 벡터: `[2, 1, -3, 2]`
- 각 단어별로 특정 layer에서 발견된 4개의 노드의 값들을 모아서 평균과 표준편차를 구한다.
  - "thinking"에 대한 인코딩 벡터의 평균과 표준편차: $\mu=3.5, \; \sigma=1.11$
  - "machines"에 대한 인코딩 벡터의 평균과 표준편차: $\mu=0.5, \; \sigma=2.06$
- 이 평균과 분산을 각각 0과 1로 만들어주는 Normalize를 수행한다.
  - normalize가 수행된 "thinking"에 대한 인코딩 벡터: `[0.65, -0.45, -1.35, 1.25]`
  - normalize가 수행된 "machines"에 대한 인코딩 벡터: `[0.7, -1.5, -0.3, 1.1]`
- 이렇게 변환된 인코딩 벡터는 특정 단어 내에서, 즉 특정 layer내에서 값들의 평균과 분산이 각각 0과 1이 되도록 만들어졌다.

<br>

**step 2 details**

- 그 다음 원하는 평균과 분산을 주입하기 위한 affine transformation을 수행할 때에는 $y=ax+b$에서 $(a, b)$의 한 세트를 각각의 단어의 같은 차원에 해당하는 값들에 적용한다.
  - 첫번째 차원: $y=3x+1$ 적용
  - 두번째 차원: $y=x+1$ 적용
  - 세번째 차원: $y=-x+0$ 적용
  - 네번째 차원: $y=-2x+2$ 적용

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15kCRLewsG0f4RPft0bo6SE5kADQfc6W1' width=800/>

<br>

## 7.8 Positional Encoding

- RNN과 달리 Self-Attention 기반으로 주어진 시퀀스를 인코딩하는 경우 각 단어 벡터에 대응하는 인코딩 벡터가 출력된다.
- 이때 시퀀스의 단어의 순서를 변경하더라도 각 단어에 대한 인코딩 벡터가 동일하게 출력될 것이다.
- 이러한 순서를 무시한다는 특성은 시퀀스읜 단어 순서 정보를 고려해서 인코딩하지 못하게 된다.
- 이러한 문제를 해결하기 위한 기법으로 사용되는 것이 Positional Encoding이다.

<br>

### 7.8.1 입력 벡터에 위치 정보를 포함

- "I go home" 이라는 시퀀스의 각 단어에 대한 입력 벡터가 3차원 벡터로 주어졌다고 하자.
  - "I": `[3, -2, 4]`
- 이때 "I"라는 단어가 전체 시퀀스에서 첫 번째 위치에 등장했다는 정보를 해당 입력 벡터에 포함시켜줘야 한다.
  - 단순하게 단어의 출현 순서에 대응하는 벡터의 차원에 1000이라는 값을 더해주게 되면 동일한 단어가 등장하더라도 서로 다른 벡터가 되게 만들 수 있다.
- 이와 같이 순서를 특정할 수 있는 unique한 상수 벡터를 각 순서에 등장하는 단어 벡터에 더해줘야 한다.
- 이것을 바로 Positinal Encoding이라고 한다.

<br>

### 7.8.2 주기 함수의 사용

- Use sinusoidal functions of different frequencies
- 더해주는 벡터를 어떻게 결정하느냐는 위에서 이야기한 단순히 1000을 더해주는 방식으로는 하지 않고 위치에 따라 구별할 수 있는 벡터를 sine과 cosine 등으로 이루어진 주기 함수를 사용한다.
- 주기가 서로 다른 함수들을 이용하여 해당 함수에서 발생된 특정 x값에서의 함수값을 모아서 위치를 나타내는 벡터로 사용한다.

<br>

- 아래와 같이 각 dimension별로 sine, cosine 함수를 만든다.
  - 첫 번째 dim: 주기가 위치 정보에 따라 변하는 sine 함수 사용
  - 두 번째 dim: 주기가 위치 정보에 따라 변하는 cosine 함수 사용
  - ...

$$
\begin{gathered}
P E_{(p o s, 2 i)}=\sin \left(p o s / 10000^{2 i / d_{\text {model }}}\right) \\
P E_{(p o s, 2 i+1)}=\cos \left(p o s / 10000^{2 i / d_{\text {model }}}\right)
\end{gathered}
$$

<br>

### 7.8.3 Positional Encoding vector의 생성

- 아래 그림과 같이 dimension 갯수만큼의 주기가 서로 다른 sine, cosine 함수들이 번갈아가면서 그래프가 생성된다.
- 이러한 패턴을 만들어 놓은 후 각 위치를 x축에 대응시켜 모든 위치에 대한 positional encoding 벡터들을 만든다.
  - ex) x=0, 즉 첫 번째 위치를 나타내는 positional encoding 벡터의 각 dimension에 해당하는 값들은 각각의 dimension별로 그려진 함수들이 x=0에 대응하는 값들로 채워진다.
  - x=0's positional encoding vector: `[..., 0, 1, 0, 1]`
  - ...
  - x=20's positional encoding vector: `[..., 0.5, -1.0, 0.75, 0.5]`
  - ...


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15kGSqoO19y4tYjxrkC18l6gV_F5I08du' width=700/>

<br>

- Easily learn to attend by relative position, since for any fixed offset $k$, $P E_{(pos + k)}$ can be represented as linear function of $PE_{(pos)}$
- 아래 그림은 주어진 sine 및 cosine 함수들을 통해 128차원의 입력 벡터를 기준으로 해서 만들어진 position encoding 벡터들을 나타낸다.
  - 그림에서 y축, 즉 각 position에 해당하는 128차원의 positional encoding 벡터의 값들을 확인할 수 있다.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15mnAw9gFx7riBEQPbAzVlLTl_yOJVkaR' width=500/>

<br>

## 7.9 Warm-up Learning Rate Scheduler

- Transformer 모델에서 사용된 learning rate 스케쥴링 기법에 대해 살펴보자.

<br>

- Adam과 같은 기법을 통해 gradient descent에 대한 최적화를 수행하는 과정에서 하이퍼파라미터로 사용되는 learning rate를 학습 전과정 동안 하나의 고정된 값으로 사용하게 된다.
- 하지만 학습을 좀 더 빠르게 하고 최종적으로 수렴한 모델의 성능을 올리기 위한 목적으로서 learning rate의 값을 학습 중에 적절하게 변경해서 사용하는 것이 learning rate 스케쥴링 기법이다.

<br>

$$
\text { learning rate } =d_{\text {model }}^{-0.5} \cdot \text { min}\left(\text{#step }^{-0.5}, \text { #step } \cdot \text { warmup_steps }^{-1.5}\right)
$$

<br>

- x축: number of iteration
- 아래 그림과 같이 초반에는 굉장히 작은 값의 learning rate를 사용한다.
  - 초반에 너무 큰 보폭이 발생하는 것을 방지
- 어느정도 iteration이 진행된 후 완만한 구간에 도달하게 됐을 때 아직 global minimum에 도달하지 못했을 수 있기 때문에 learning rate의 값을 iteration 수에 비례해서 증가시켜 보폭을 증가시킨다.
- 큰 보폭을 통해 local minimum을 빠져나와 global minimum 근처에 도달한 경우 다시 learning rate을 감소시켜 global minimum에 도달할 수 있게 한다.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15u0Q9L0toLHWXPXvu1sM-WFS_Z-5hQGw' width=500/>

- 출처: Group Normalization, ECCV’18

<br>

## 7.10 Encoder Process Review

- 지금까지의 설명을 통해 Transformer의 인코딩 과정에 대해 알아보았다.
- 각 인코딩 블럭을 6, 12, 24 단위로 쌓는다.
  - 각 인코딩 블럭의 파라미터들을 다른 인코딩 블럭의 파라미터들과 독립적이다.
- 최종적으로 모든 인코딩 블럭을 통과한 각 단어에 대한 인코딩 벡터를 얻게 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15E_Jt0eftyRQ2-kTn-7-nm52zbiF2kkm' width=800/>

- 출처: http://jalammar.github.io/illustrated-transformer/

<br>

### 7.10.1 Encoder Self-Attention Visualization

- 인코더 모듈이 보여주는 흥미로운 패턴들에 대해 알아보자.

<br>

- Words start to pay attention to oher words in sensible ways
- [https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb](https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb)

<br>

- 아래 그림에서 위쪽 문장과 아래쪽 문장은 동일한 문장이다.
- 위쪽 문장의 단어들이 query 벡터로 사용되어 Attention 모듈을 수행했을 때 아래 문장의 단어들 중 어떤 단어의 정보를 가져가는 지를 시각화한 것이다.
  - "making" 이라는 단어가 query 벡터로 사용됐을 때 자기자신("making")에 대한 정보를 가져가고 있는 것을 볼 수 있고, "more", "difficult" 단어의 정보를 많이 가져가는 것을 볼 수 있다.
  - 또한 "2009" 라는 단어에 대한 정보도 가져가는 모습을 보인다.
- 아래 그림에서 화살표의 두께와 색깔 뿐만 아니라 하나의 단어에 대해서도 여러 grid 형태를 나타내는 것을 볼 수 있다.
  - grid의 가장 위쪽 줄이 첫 번째 head의 attention pattern을 나타내고, 두 번째 줄이 두 번째 head의 attention pattern을 의미한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15w_L080q9I_H6sPBP95MmQbN1ehKn-fM' width=600/>

<br>

- 또 다른 단어 "its"를 query로 사용했을 때 Attention의 시각화 패턴을 보면 "Law"와 "application"에 대해 첫 번째 head에 대해서는 "Law"라는 단어에 많은 가중치를 두고 있고, 두 번째 head에 대해서는 "application"에 많은 가중치를 두고 있는 모습을 볼 수 있다.
- 이를 통해 첫 번째 head에서는 "its"라는 단어가 가리키는 단어에 대한 정보를 담고 있다는 것을 일 수 있고, 두 번째 head에서는 "its"라는 단어가 한정해주는 명사에 대한 정보를 담고 있는 것이라고 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=167I30XxxA5ZO5KwQZi4P8Pj2qbhAKCsk' width=600/>

<br>

## 7.11 Decoder

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16CoADBhSKc99ulLqkAtI7PMozFf189yW' width=300/>

<br>

### 7.11.1 Decoder Process

- 디코더는 ground truth에 해당하는 문장의 단어들을 오른쪽으로 한칸씩 이동(shifted right)시킨 시퀀스를 입력으로 받는다.
  - task의 목적: `I go home` -> `나는 집에 간다`
  - shifted right sequence: `<sos> 나는 집에` (디코더의 입력 시퀀스)

<br>

- 이 입력 벡터는 Positional Encoding을 거친 후 **Masked Multi-Head Attention**의 입력으로 사용되고 Attention 모듈의 output이 Add & Norm 과정을 거쳐 인코딩 벡터들이 만들어진다.
  - *이 과정은 Seq2seq w/ attention 에서 디코더의 hidden state vector를 만들어내는 과정으로 이해할 수 있다.*
  - 여기서의 Attention 모듈에는 "Masked" 라는 단어가 포함되어 있는 것을 볼 수 있다. (뒤에서 설명)

<br>

- 이렇게 만들어진 인코딩 벡터들은 다음 **Multi-Head Attention**의 입력으로 전달되어 Attention 모듈이 수행된다.
  - <font color='skyblue'>Masked Multi-Head Attention에서 만들어진 인코딩 벡터</font>들은 해당 Attention 모듈에서는 <font color='skyblue'>Query 벡터를 만드는데만 사용</font>된다.
  - <font color='orange'>Key 벡터와 Value 벡터</font>는 <font color='orange'>인코더의 최종 output 벡터를 사용</font>하여 만들어진다.
  - *이는 Seq2seq w/ attention의 Attention 모듈에서 디코더의 hidden state vector를 query 벡터로 사용하여 입력으로 받고 인코더의 hidden state vector들을 key 벡터로서 입력으로 받아 가중치를 구하는 과정에 해당한다고 볼 수 있다.*

<br>

- 그 다음 Add & Norm를 수행하고, 추가적으로 FFN와 Add & Norm 과정을 통해 입력 시퀀스(`<sos> 나는 집에`)의 각 단어에 다음 단어를 예측하는 디코더의 최종 인코딩 벡터가 출력된다.
  - 이 디코더의 최종 인코딩 벡터는 디코더의 입력 문장에 대한 정보를 갖고 있으면서 동시에 인코더에서 주어진 벡터에 대한 정보 또한 가져와서 다양한 정보가 결합된 벡터가 될 것이다.

<br>

- 최종적으로 디코더의 출력에 대해 Linear Transformation을 수행하여 Target Language(한국어)의 vocab size에 해당하는 벡터를 생성한다.
- 여기에 softmax를 취하고 특정한 target language의 단어에 대한 확률값을 뽑아서 다음에 나올 단어를 예측하게 된다.

<br>

- 이렇게 예측한 target word에 대한 확률분포는 ground truth 단어와의 softmax loss를 통해서 backpropagation을 통해 전체 네트워크가 학습되게 된다.

<br>

### 7.11.2 Changed sub-layer

- Two sub-layer changes in decode

<br>

#### 7.11.2.1 Masked Multi-Head Attention

- Masked decoder self-attention on previously generated output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16FdWj7vX1qs80AVw6vOiP3vn9U50RAgT' width=300/>

<br>

#### 7.11.2.2 Encoder-Decoder Attention

- queries come from previous decoder layer and keys and values come from output of encoder

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16G1LG_JWIGhnKgCD-NrZ2ZiJ8QFmYdQk' width=300/>

<br>

### 7.11.3 Masked Self-Attention

- Masked Self-Attention의 기본 아이디어는 디코딩 과정 중에 입력으로서 shifted right된 시퀀스를 입력으로 받고 Self-Attention을 통해 시퀀스의 각 단어를 인코딩하는 과정에서 <font color='skyblue'>정보에 대한 접근 가능 여부</font>와 관련이 된다.

<br>

- Those words not yet generated cannot be accessed during the inference time
- Renormalization of softmax output prevents the model from accessing ungenerated words

<br>

- 아래 그림의 오른쪽 부분과 같이 `<sos> 나는 집에` 라는 시퀀스의 단어들에 대해 Self-Attention을 수행하는 경우 $Q K^T$의 계산 결과로서 각 단어들 간의 유사도 정보를 갖고 있는 행렬을 얻게 된다.
  - 단어가 3개이므로 이 행렬의 크기는 3x3이 된다.
- 각 단어가 나머지 모든 단어에 대해서 모든 정보에 대한 접근을 허용하게 되면 예측 과정에서 학습이 제대로 이뤄지지 않을 수 있다.
  - 첫 번째 time step에서 `<sos>` 단어까지만 정보가 주어졌을 때 디코더 학습 과정에서 뒤에 `나는`과 `집에`라는 단어가 주어져야 한다는 것은 알고 있다.
  - 하지만 학습 과정에서는 뒤에 등장하는 단어에 대한 정보 없이 다음에 나올 단어를 예측할 수 있어야 한다.
    - `<sos>` 라는 단어만을 가지고 `나는`이라는 단어를 예측해야 한다.
    - `<sos>`와 `나는` 이라는 단어만을 가지고 `집에`라는 단어를 예측해야 한다.
  - 즉, 배치 프로세싱에 위해서 입력을 동시에 주어지기는 하지만 <font color='skyblue'>뒤에 나오는 단어들에 대한 정보는 제외</font>를 해줘야 한다.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16JpSnkhbD3TUyZdEP4Y9woI6yjqzAqQ6' width=800/>

- 아래 그림과 같이 $Q K^T$ 의 결과 행렬에 row wise softmax가 적용된 이후에 확률값이 계산된 경우에는 뒤에 등장하는 단어의 정보를 가져오는 것을 차단해줘야 한다.
  - `<sos>`의 경우 뒤에서 등장하는 단어 `나는`, `집에` 에 해당하는 확률 0.05와 0.04에 대한 정보를 가져오는 것을 차단해야 한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16LYt0qWyz9A0Qpa3YmYB-CKAOsyUM7i4' width=800/>

- 그래서 뒤에 등장하는 정보에 해당하는 확률값들을 0으로 바꿔주는 후처리를 진행해줘야 한다.
  - 정방 행렬 형태의 행렬에 대해 대각선 원소 위쪽에 있는 정보들을 0으로 바꿔준다.
- 그런 다음 row 단위로 다시 합이 1이 되도록 normalize를 하게 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16Put7lhCcn8mQgpjEmK-5WEN7zRap7TV' width=800/>

<br>

- Masked Self-Attention은 Attention을 모든 단어가 서로에 대한 정보를 볼수 있게 한 후 후처리적으로 보면 안되는, 즉 뒤에 등장하는 단어에 대한 Attention 가중치를 0으로 만들어주는 작업을 수행한 후 value 벡터와 가중 평균을 내는 방식으로 진행되는 디코더에서 사용되는 Attention의 변형이다.

<br>

## 7.12 Experimental Results

- Results on English-German/French translation (newstest2014)
- 자연어 처리에서 가장 활발하게 연구되고 있는 기계번역 분야에 Transformer 모델이 처음 제안됐고, 이전까지 가장 좋은 성능을 내던 Seq2seq w/ Attention 보다 더 좋은 성능을 보였다.
- 각 모델별로 BLEU 와 학습 시 걸리는 시간(Training Cost)을 비교해봤을 때 Transformer 기반의 모델이 가장 좋은 성능을 나타내는 것을 확인할 수 있다.
- 하지만 가장 좋은 BLEU 스코어가 20~40%의 값을 가지고 있는 데 BLEU 스코어의 최대값이 100%인 것을 감안해보면 아직 낮은 수준에 머문다는 것을 알 수 있다.
  - 그렇지만 보기보단 좋은 성능을 내는 결과에 해당한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16UWMsuw7LUgPiHOI6slAvmnu_Qamv6vv' width=700/>